In [ ]:
import torch
import transformers
import vllm

/home/rkhanm1/anaconda3/envs/vllm_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 02-14 11:09:11 __init__.py:190] Automatically detected platform cuda.


2025-02-14 11:09:11,803	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
import torch
from vllm import LLM, SamplingParams

# Load the model (adjust the name if needed)
model_name = "meta-llama/Llama-3.2-1B-Instruct"
llm = LLM(model=model_name, gpu_memory_utilization=0.5, enforce_eager=True, \
          tensor_parallel_size=2)
llm

INFO 02-14 11:09:58 config.py:542] This model supports multiple tasks: {'classify', 'score', 'reward', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 02-14 11:09:58 config.py:1401] Defaulting to use mp for distributed inference
WARNING 02-14 11:09:58 arg_utils.py:1135] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 02-14 11:09:58 config.py:1556] Chunked prefill is enabled with max_num_batched_tokens=2048.
WARNING 02-14 11:09:58 cuda.py:95] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 02-14 11:09:58 config.py:678] Async output processing is not supported on the current platform type cuda.
INFO 02-14 11:09:58 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: mode

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(VllmWorkerProcess pid=171488) INFO 02-14 11:10:05 weight_utils.py:252] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.51it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.49it/s]


(VllmWorkerProcess pid=171488) INFO 02-14 11:10:05 weight_utils.py:297] No model.safetensors.index.json found in remote.
INFO 02-14 11:10:05 model_runner.py:1115] Loading model weights took 1.1667 GB


(VllmWorkerProcess pid=171488) INFO 02-14 11:10:05 model_runner.py:1115] Loading model weights took 1.1667 GB
INFO 02-14 11:10:09 worker.py:267] Memory profiling takes 3.10 seconds
INFO 02-14 11:10:09 worker.py:267] the current vLLM instance can use total_gpu_memory (39.39GiB) x gpu_memory_utilization (0.50) = 19.70GiB
INFO 02-14 11:10:09 worker.py:267] model weights take 1.17GiB; non_torch_memory takes 1.54GiB; PyTorch activation peak memory takes 1.18GiB; the rest of the memory reserved for KV Cache is 15.81GiB.
(VllmWorkerProcess pid=171488) INFO 02-14 11:10:09 worker.py:267] Memory profiling takes 3.11 seconds
(VllmWorkerProcess pid=171488) INFO 02-14 11:10:09 worker.py:267] the current vLLM instance can use total_gpu_memory (39.39GiB) x gpu_memory_utilization (0.50) = 19.70GiB
(VllmWorkerProcess pid=171488) INFO 02-14 11:10:09 worker.py:267] model weights take 1.17GiB; non_torch_memory takes 1.54GiB; PyTorch activation peak memory takes 0.08GiB; the rest of the memory reserved for

In [4]:
# Define the prompt
prompt = "What is the capital of France?"

# Set sampling parameters
sampling_params = SamplingParams(temperature=0.7, top_p=0.9, max_tokens=100)

# Run inference
output = llm.generate([prompt], sampling_params)
print("Generated Output:", output[0].outputs[0].text)

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.74s/it, est. speed input: 2.92 toks/s, output: 36.56 toks/s]

Generated Output:  Paris.
The capital of France is indeed Paris. It is the most famous city in France and is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum.

Paris is a city that is steeped in history and culture, and it is a popular destination for tourists from all over the world. The city has a rich and vibrant atmosphere, with a diverse range of cultures, cuisines, and lifestyles.

Some interesting facts about Paris include:




In [6]:
# Define the prompt (asked 100 times in a batch)
prompt = "What is the capital of France?"
batch_prompts = [prompt] * 1000  # Create a batch of 100 identical prompts

# Set sampling parameters
sampling_params = SamplingParams(temperature=0, top_p=0.9, max_tokens=100)

# Run batched inference
outputs = llm.generate(batch_prompts, sampling_params)

# Print results
for i, output in enumerate(outputs):
    print(f"Response {i+1}: {output.outputs[0].text}")

Processed prompts: 100%|██████████| 1000/1000 [00:08<00:00, 118.95it/s, est. speed input: 951.61 toks/s, output: 10586.68 toks/s]

Response 1:  Paris.
The capital of France is Paris. Paris is the most populous city in France and is known for its rich history, art, fashion, and cuisine. It is also home to many famous landmarks such as the Eiffel Tower, Notre Dame Cathedral, and the Louvre Museum. Paris is a popular tourist destination and is often referred to as the "City of Light" due to its association with the Enlightenment and the French Revolution.
Response 2:  Paris.
The capital of France is Paris. Paris is the most populous city in France and is known for its rich history, art, fashion, and cuisine. It is also home to many famous landmarks such as the Eiffel Tower, Notre Dame Cathedral, and the Louvre Museum. Paris is a popular tourist destination and is often referred to as the "City of Light" due to its association with the Enlightenment and the French Revolution.
Response 3:  Paris.
The capital of France is Paris. Paris is the most populous city in France and is known for its rich history, art, fashion, a

In [8]:
conversation = [
    {
        "role": "system",
        "content": "You are a helpful assistant"
    },
    {
        "role": "user",
        "content": "what is the capital of France?"
    }
]
conversations = [conversation for _ in range(1000)]

# We turn on tqdm progress bar to verify it's indeed running batch inference
outputs = llm.chat(messages=conversations,
                   sampling_params=sampling_params,
                   use_tqdm=True)
# Print results
for i, output in enumerate(outputs):
    print(f"Response {i+1}: {output.outputs[0].text}")

Processed prompts: 100%|██████████| 1000/1000 [00:01<00:00, 899.04it/s, est. speed input: 43159.56 toks/s, output: 7193.21 toks/s]

Response 1: The capital of France is Paris.
Response 2: The capital of France is Paris.
Response 3: The capital of France is Paris.
Response 4: The capital of France is Paris.
Response 5: The capital of France is Paris.
Response 6: The capital of France is Paris.
Response 7: The capital of France is Paris.
Response 8: The capital of France is Paris.
Response 9: The capital of France is Paris.
Response 10: The capital of France is Paris.
Response 11: The capital of France is Paris.
Response 12: The capital of France is Paris.
Response 13: The capital of France is Paris.
Response 14: The capital of France is Paris.
Response 15: The capital of France is Paris.
Response 16: The capital of France is Paris.
Response 17: The capital of France is Paris.
Response 18: The capital of France is Paris.
Response 19: The capital of France is Paris.
Response 20: The capital of France is Paris.
Response 21: The capital of France is Paris.
Response 22: The capital of France is Paris.
Response 23: The ca